Run the first notebook so I can work with the results from it in this one

In [234]:
%run ./Toronto_Scraping.ipynb foo()
print('Ready')

Libraries imported.


./Toronto_Scraping.ipynb:2: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 2 of the file ./Toronto_Scraping.ipynb. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  "cells": [


Ready


Import tools/libraries I'll be using for this section

In [235]:
import time
import json
from pandas.io.json import json_normalize
from geopy.geocoders import Nominatim
import folium # map rendering library
import itertools
import requests

print('Libraries imported.')

Libraries imported.


Geocode Toronto, will be used for map targeting

In [236]:
address = 'Toronto, Ontario'
geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


<p>Create blanks for the dataframe's latitude and longitude columns and a dictionary of the data I'll map to those columns after geocoding</p>
<p>Not elegant, definitely can be done simpler I'm sure, but this works for me at the moment</p>

In [237]:
df['Latitude'] = None
df['Longitude'] = None
coords = {}

Geocoding with LocationIQ: https://locationiq.com/

In [243]:
# *************** token = '2f366e9408ec04' ***************** DELETE ME
for i, row in df.iterrows():
    pcode = row[0]
    url = "https://us1.locationiq.com/v1/search.php"
    data = {
            'key': 'SECRET, SET UP YOUR OWN LOCATIONIQ ACCOUNT AND INPUT YOUR KEY HERE TO USE',
            'postalcode': pcode,
            'format': 'json',
            'viewbox': '-79.747207, 43.293963, -79.027207, 44.013963',
            'bounded': '1',
            'limit': '1',
            'accept-language': 'en'
            }
    if pcode in coords.keys():
        continue
    else:
        try:
            json = requests.get(url, params=data).json()
            lat = float(json[0]['lat'])
            lon = float(json[0]['lon'])
        except:
            print('Error geocoding', pcode)
            print(json)
            continue
    coords[pcode] = [lat, lon] # Formatted like this as there are a couple duplicates, can run split, comma delim, to grab bits so nbd
    time.sleep(1.5) # was hitting a query-per-second limit

print('Geocoding complete. Returned', len(coords), 'results.')

Geocoding complete. Returned 103 results.


<p>Error geocoding M7R, tried doing it manually on LocationIQ but still failed</p>
<p>Solved by pulling from the provided dataset</p>
<p>This is concerning, in any case</p>

In [244]:
# Had an error with Caledonia-Fairbanks - running low on time so gonna just do this sloppily rather than
# try playing around more with the above code

coords['M7R'] = [43.6369656, -79.615819]

Making sure everything works, no duplicates, right size, etc

In [245]:
# checking

nondupes = []
dupes = []

for k, v in coords.items():
    
    if not k in nondupes and not v in nondupes:
        nondupes.append(i)
    else:
        dupes.append(i)
        print('Duplicate found:',k,v)
        
print(len(coords))
print(len(nondupes))
print(len(dupes))

103
103
0


Mapping the coordinates to the original dataframe

In [275]:
lats = {}
lons = {}
for k, v in coords.items():
    lats[k] = v[0]
    lons[k] = v[1]
df["Latitude"] = df["Postcode"].map(lats)
df["Longitude"] = df["Postcode"].map(lons)

Let's load in the rest of the data provided just in case, since it looked a bit different from what I got from my geocoding

In [276]:
df2 = pd.read_csv('http://cocl.us/Geospatial_data', header=0, names=['Postcode', 'Latitude', 'Longitude'])
df2.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Let's put both of em on the map and see just how big that difference is

In [250]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)
for lat, lng, borough, postcode in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Postcode']):
    label = '{}, {}'.format(postcode, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
#        popup=label, not sure why but the map only shows one circle when I un-comment. Same in the lab with the provided code... hm
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
for lat, lng, postcode in zip(df2['Latitude'], df2['Longitude'], df2['Postcode']):
    label = '{}'.format(postcode)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=2.5,
#        popup=label, not sure why but the map only shows one circle when I un-comment. Same in the lab with the provided code... hm
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)

In [251]:
map_toronto

<p>Damn, wellp, let's just roll with the provided data in this case, seems there's a lot of discrepancies</p>
<p>Let's add the alternative latlons to the dataframe and create a new dataframe of the new data to work with going forward</p>

In [283]:
df["Latitude_Alt"] = df["Postcode"].map(df2.set_index('Postcode')['Latitude'])
df["Longitude_Alt"] = df["Postcode"].map(df2.set_index('Postcode')['Longitude'])
df_alt = df.copy()
df_alt['Latitude'] = df['Latitude_Alt']
df_alt['Longitude'] = df['Longitude_Alt']
df_alt.drop(['Latitude_Alt', 'Longitude_Alt'], axis=1, inplace=True)
df_alt.shape

(103, 5)

And check

In [286]:
df_alt.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [285]:
#df_alt

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848
